In [110]:
# Time seris

# Time series study


In [111]:
import pandas as pd
import numpy as np

In [112]:
dates = pd.date_range(start = '2016', periods = 4, freq = 'Q')
data = range(1,5)
quarterly = pd.Series(data = data, index = dates)
quarterly

2016-03-31    1
2016-06-30    2
2016-09-30    3
2016-12-31    4
Freq: Q-DEC, dtype: int64

Vamos fazer um upsample e mudar a frequencia da nossa time series de quartely para monthly.

In [113]:
monthly = quarterly.asfreq('M')
monthly

2016-03-31    1.0
2016-04-30    NaN
2016-05-31    NaN
2016-06-30    2.0
2016-07-31    NaN
2016-08-31    NaN
2016-09-30    3.0
2016-10-31    NaN
2016-11-30    NaN
2016-12-31    4.0
Freq: M, dtype: float64

In [114]:
# Vamos converter a series monthly em um dataframe e avaliar as possibilidades de preenchimento dos dados faltantes

In [115]:
monthly = monthly.to_frame('baseline')
monthly

,baseline
2016-03-31,1.0
2016-04-30,NaN
2016-05-31,NaN
2016-06-30,2.0
2016-07-31,NaN
2016-08-31,NaN
2016-09-30,3.0
2016-10-31,NaN
2016-11-30,NaN
2016-12-31,4.0


In [116]:
monthly['ffill'] = quarterly.asfreq('M', method = 'ffill')
monthly['bfill'] = quarterly.asfreq('M', method = 'bfill')
monthly['value'] = quarterly.asfreq('M', fill_value=0)
monthly


,baseline,ffill,bfill,value
2016-03-31,1.0,1,1,1
2016-04-30,NaN,1,2,0
2016-05-31,NaN,1,2,0
2016-06-30,2.0,2,2,2
2016-07-31,NaN,2,3,0
2016-08-31,NaN,2,3,0
2016-09-30,3.0,3,3,3
2016-10-31,NaN,3,4,0
2016-11-30,NaN,3,4,0
2016-12-31,4.0,4,4,4


Podemos fazer exatamente a mesma coisa que fizemos com .asfreq() usando .reindex

# Rolling windows

É muito importante usar parse_dates e index_col em pd.read_csv. Veja a diferença:

In [117]:
stocks = pd.read_csv('Google_Stock_Price_Train.csv')
stocks.head(3)

,Date,Open,High,Low,Close,Volume
0,1/3/2012,325.25,332.83,324.97,663.59,"7,380,500"
1,1/4/2012,331.27,333.87,329.08,666.45,"5,749,400"
2,1/5/2012,329.83,330.75,326.89,657.21,"6,590,300"


In [118]:
stocks = pd.read_csv('Google_Stock_Price_Train.csv', parse_dates=[0], index_col = [0])
stocks.head(3)


,Open,High,Low,Close,Volume
Date,,,,,
2012-01-03,325.25,332.83,324.97,663.59,"7,380,500"
2012-01-04,331.27,333.87,329.08,666.45,"5,749,400"
2012-01-05,329.83,330.75,326.89,657.21,"6,590,300"


Entretanto, verificamos que as últimas duas colunas estão como 'object' e não float:

In [119]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2012-01-03 to 2016-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1258 non-null   float64
 1   High    1258 non-null   float64
 2   Low     1258 non-null   float64
 3   Close   1258 non-null   object 
 4   Volume  1258 non-null   object 
dtypes: float64(3), object(2)
memory usage: 59.0+ KB


Ler as ultimas duas colunas como float não está funcionando. Veja:

In [120]:
stocks = pd.read_csv('Google_Stock_Price_Train.csv', parse_dates=[0], index_col = [0], dtype={'Close': np.float64, 'Volume': np.float64})
stocks.head(3)


ValueError: could not convert string to float: '1,008.64'

Não há nada de errado com a coluna Volume, aparentemente:

In [121]:

stocks[stocks['Volume'].str.contains('.', regex = False)]

,Open,High,Low,Close,Volume
Date,,,,,


Mas o pd.read_csv não entende as vírgulas na coluna 'Close'.

In [122]:
control_subset = stocks[stocks['Close'].str.contains(',')].copy()
control_subset

,Open,High,Low,Close,Volume
Date,,,,,
2013-10-18,486.47,505.83,485.18,"1,008.64","23,219,400"
2013-10-21,503.84,507.60,497.91,"1,000.55","7,283,700"
2013-10-22,500.62,504.61,496.04,"1,004.24","4,433,700"
2013-10-23,498.63,515.44,498.45,"1,028.59","5,346,300"
2013-10-24,514.01,518.34,510.49,"1,022.74","4,199,700"
...,...,...,...,...,...
2014-03-20,597.73,602.55,595.45,"1,193.88","3,381,900"
2014-03-21,600.90,602.56,589.02,"1,179.80","6,441,000"
2014-03-24,589.88,590.24,570.84,"1,154.76","6,096,800"


Vamos tirar as vírgulas da coluna "close":

In [123]:
stocks[['Close', 'Volume']]

,Close,Volume
Date,,
2012-01-03,663.59,"7,380,500"
2012-01-04,666.45,"5,749,400"
2012-01-05,657.21,"6,590,300"
2012-01-06,648.24,"5,405,900"
2012-01-09,620.76,"11,688,800"
...,...,...
2016-12-23,789.91,"623,400"
2016-12-27,791.55,"789,100"
2016-12-28,785.05,"1,153,800"


In [124]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2012-01-03 to 2016-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1258 non-null   float64
 1   High    1258 non-null   float64
 2   Low     1258 non-null   float64
 3   Close   1258 non-null   object 
 4   Volume  1258 non-null   object 
dtypes: float64(3), object(2)
memory usage: 59.0+ KB


In [125]:
stocks['Close'] = stocks['Close'].str.replace(',', repl = '')
#stocks['Volume'] = stocks['Volume'].str.replace(',', repl = '')
stocks[stocks == control_subset]

ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects

In [ ]:
control_index = list(control_subset.index)
stocks[stocks.index.isin(control_index)]

,Open,High,Low,Close,Volume
Date,,,,,
2013-10-18,486.47,505.83,485.18,1008.64,"23,219,400"
2013-10-21,503.84,507.60,497.91,1000.55,"7,283,700"
2013-10-22,500.62,504.61,496.04,1004.24,"4,433,700"
2013-10-23,498.63,515.44,498.45,1028.59,"5,346,300"
2013-10-24,514.01,518.34,510.49,1022.74,"4,199,700"
...,...,...,...,...,...
2014-03-20,597.73,602.55,595.45,1193.88,"3,381,900"
2014-03-21,600.90,602.56,589.02,1179.8,"6,441,000"
2014-03-24,589.88,590.24,570.84,1154.76,"6,096,800"


In [ ]:
stocks.loc[:, 'Close'] = stocks.loc[:, 'Close'].astype(float)
stocks.info()

NameError: name 'float64' is not defined

In [ ]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2012-01-03 to 2016-12-30
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    1258 non-null   float64
 1   High    1258 non-null   float64
 2   Low     1258 non-null   float64
 3   Close   1258 non-null   object 
 4   Volume  1258 non-null   object 
dtypes: float64(3), object(2)
memory usage: 59.0+ KB


In [ ]:
stocks = stocks.astype(float)

ValueError: could not convert string to float: '1,008.64'

## Window functions

In [ ]:
rolling = stocks.Close.rolling('360D')
#q10 = rolling.quantile(0.1).to_frame('q10')
median = rolling.median()

DataError: No numeric types to aggregate